# Exam Scheduling
After spending all spring break chasing Easter eggs, the faculty professors no longer have time to schedule exams. You and your peers fear that this will result in a disastrous result.

Students are never happy studying for exams. However, they are even less happy when they have to take two exams on the same day.
Willing to show your mastery of propositional logic, you decide to compute a schedule to submit to the professors that will make everyone happy.

You are given a list of students and the exams they have to take. Your task is to find a satisfactory schedule using a SAT solver. That is, a schedule such that every exam is planned in a room with enough capacity for all students taking the exam while avoiding multiple exams on the same day. In other words:
- each exam is planned exactly once, in a room with enough capacity for all students taking the exam;
- a room can host at most one exam at a time;
- no student has two exams on the same day.

## Rules
You have to:
- implement the functions `encode_cnf` and `decode_model` in the "exam_scheduler.py" file;
- use the provided template;
- briefly describe your encoding in the python file;
- submit the "exam_scheduler.py" python file with your solution.
- submit a singled file name "proj2.[pdf/txt/md]" with a brief description of your solution. Evaluate how your approach scales with the number of students and exams. Estimate the number of variables and clauses that your encoding generates depending on the number of students, days, courses, and rooms. This document should not be more than two pages long (except if you addressed the pigeonhole problem).

You may:
- write any additional function you need;
- use global variables, while it is not recommended, it is allowed to make your code cleaner.

You may not:
- use any external library (except for the ones already imported in the template);
- print anything on the standard output and standard error in your final solution;
- modify the signature of the functions (the automatic tests will fail otherwise).

You are advised to:
- test your program;
- keep your code clean and readable;
- write comments to explain your reasoning;
- experiment corner cases.

## Warning
We will use some automatic tests as sanity checks for your implementation. If you do not respect the output format or your code throws an error, you will get 0 points for the failed tests. Make sure to test your implementation.

In [ ]:
%pip install python-sat -q
%pip install pandas -q

# this should install the required packages
# In case it does not work, remove the -q flag to see the error message

In [ ]:
# The teaching assistant used python 3.12.03 to run this code. If you have any problem with your version, please first try to run the code with python 3.12.03.
# Those are the only imports you are allowed to use.
from pysat.formula import CNF
from pysat.solvers import Solver
from pandas import read_csv
import os
import time

In [ ]:
# Here select the instance you want to solve
# You can generate your own instances using the same format as the one provided
# uncomment the code below to load the instance

n_rooms = 2
n_days = 3
n_courses = 5
n_students = 10
unsat = False

# n_rooms = 3
# n_days = 10
# n_courses = 25
# n_students = 100
# unsat = False

# n_rooms = 5
# n_days = 5
# n_courses = 20
# n_students = 50
# unsat = True

In [ ]:
# You should not have change anything below this line
# You may want to change it to test your code, but beware not to change the interface with the encode_cnf and decode_model functions

folder = str(n_rooms) + "_rooms_" + str(n_days) + "_days_" + str(n_courses) + "_courses_" + str(n_students) + "_students" + ("_unsat" if unsat else "") + "/"
if not os.path.exists(folder):
    print("Folder \"" + folder + "\" does not exist.")
    exit(1)

# List of students and the courses they are taking
# this is a pandas DataFrame with columns "Student" and "Courses" where "Courses" is a list of courses that the student is taking
students_courses = read_csv(folder + "students_courses.csv")
# the columns "Courses" is a list of courses that the student is taking => convert it to a list
students_courses["Courses"] = students_courses["Courses"].apply(eval)

# Select the list of unique courses taken by the students
courses = list(students_courses["Courses"].explode().unique())
# list of rooms with their capacity
# this is a pandas DataFrame with columns "Room" and "Capacity"
rooms = read_csv(folder + "rooms.csv")
# Allowed time slots to schedule the exams
days = list(read_csv(folder + "days.csv")["Day"])


In [ ]:
from exam_scheduler import encode_cnf, decode_model
from scheduler_utils import check_schedule, print_schedule

In [ ]:
# This calls your function to encode the problem into a SAT instance
cnf_list = encode_cnf(days, courses, rooms, students_courses)
print("Number of clauses: ", len(cnf_list))
# transform the list of clauses into a CNF object
cnf = CNF(from_clauses=cnf_list)
solver = Solver(bootstrap_with=cnf)
print("")
# solve the SAT instance
start = time.time()
if (not solver.solve()):
    end = time.time()
    print("No solution found")
else:
    end = time.time()
    model = solver.get_model()
    schedule = decode_model(model, days, courses, rooms, students_courses)
    print_schedule(schedule, days, courses, rooms)
    # check if the schedule is valid
    if(check_schedule(schedule, days, courses, rooms, students_courses)):
        print("Schedule is valid")
    else:
        print("Schedule is invalid")

print("Number of propagations: ", solver.accum_stats()["propagations"])
print("Time spent in solving: ", round(end - start, 5), " seconds")

# Compare with the teaching assistant
The section below gives you some reference on the efficiency of your solution. If you test your encoding on the provided test cases, you can compare your results with the ones of the teaching assistant. The TA's solution is not necessarily the best one, but it is a good reference point.

If your encoding is better or equivalent to the TA while remaining sound, you will get full points. Further improvements are for your own satisfaction.

In [ ]:
# this can give you an idea of the computation time taken by the encoding of the teaching assistant
times_TA = {
  (2, 3, 5, 10, False): 0.00015,
  (3, 10, 25, 100, False): 0.0008,
  (5, 5, 20, 50, True): 0.00092
}

n_clauses_TA = {
  (2, 3, 5, 10, False): 113,
  (3, 10, 25, 100, False): 13062,
  (5, 5, 20, 50, True): 6064
}

time_TA = times_TA.get((n_rooms, n_days, n_courses, n_students, unsat), None)
if time_TA is not None:
    print("Time of the TA: ", time_TA, " seconds")
    if (end - start) * 0.9 > time_TA :
        print("Your code is slower than the TA's code")
    elif (end - start) * 1.1 < time_TA:
        print("Your code is faster than the TA's code")
    else:
        print("Your code is as fast as the TA's code")

n_clauses = n_clauses_TA.get((n_rooms, n_days, n_courses, n_students, unsat), None)
if n_clauses is not None:
    print("Number of clauses of the TA: ", n_clauses)
    if len(cnf_list) * 0.9 > n_clauses :
        print("Your code generates more clauses than the TA's code")
    elif len(cnf_list) * 1.1 < n_clauses:
        print("Your code generates less clauses than the TA's code")
    else:
        print("Your code generates as many clauses as the TA's code")